In [ ]:
import pandas as pd

In [ ]:
# Read the CSV file into a DataFrame
teams_stats = pd.read_csv('../basketballPlayoffs/teams_stats_encoded.csv')

In [32]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
import tensorflow as tf
import numpy as np

# Split data into features and target
X = teams_stats.drop(columns=['playoff']).values.astype(np.float32)
y = teams_stats['playoff'].values.astype(np.float32)

# Define a function for creating the TensorFlow model
def create_model(input_dim, hidden_dim, num_layers):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(input_dim,)))
    model.add(tf.keras.layers.Dense(hidden_dim, activation='relu'))
    for _ in range(num_layers):
        model.add(tf.keras.layers.Dense(hidden_dim, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

# Define a function for training the model
def train_model(model, X_train, y_train, num_epochs, batch_size, lr):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=0)
    
    return model

# Define hyperparameter search space
hidden_dims = [32, 64]
num_layers_values = [1, 2]
num_epochs_values = [10, 20]
batch_sizes = [32, 64]
learning_rates = [0.001, 0.01]

best_accuracy = 0
best_model = None
best_hyperparameters = None

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for hidden_dim in hidden_dims:
    for num_layers in num_layers_values:
        for num_epochs in num_epochs_values:
            for batch_size in batch_sizes:
                for lr in learning_rates:
                    accuracies = []

                    for train_idx, test_idx in kf.split(X):
                        X_train, X_test = X[train_idx], X[test_idx]
                        y_train, y_test = y[train_idx], y[test_idx]

                        input_dim = X_train.shape[1]
                        model = create_model(input_dim, hidden_dim, num_layers)

                        trained_model = train_model(model, X_train, y_train, num_epochs, batch_size, lr)

                        y_pred = (trained_model.predict(X_test) > 0.5).astype(int).flatten()

                        accuracy = accuracy_score(y_test, y_pred)
                        accuracies.append(accuracy)

                    avg_accuracy = np.mean(accuracies)
                    std_dev = np.std(accuracies)

                    print(f"Model with hidden_dim={hidden_dim}, num_layers={num_layers}, num_epochs={num_epochs}, batch_size={batch_size}, lr={lr}")
                    print("Average Accuracy:", avg_accuracy)
                    print("Standard Deviation:", std_dev)

                    if avg_accuracy > best_accuracy:
                        best_accuracy = avg_accuracy
                        best_std_dev = std_dev
                        best_model = trained_model
                        best_hyperparameters = {
                            'hidden_dim': hidden_dim,
                            'num_layers': num_layers,
                            'num_epochs': num_epochs,
                            'batch_size': batch_size,
                            'lr': lr
                        }

print("Best Model Average Accuracy:", best_accuracy)
print("Best Model Average STD:", best_std_dev)
print("Best Hyperparameters:", best_hyperparameters)



1/1 [==============================] - 0s 41ms/step
Model with hidden_dim=32, num_layers=1, num_epochs=10, batch_size=32, lr=0.001
Average Accuracy: 0.5724358974358974
Standard Deviation: 0.12846288192142577
1/1 [==============================] - 0s 37ms/step
Model with hidden_dim=32, num_layers=1, num_epochs=10, batch_size=32, lr=0.01
Average Accuracy: 0.5865384615384615
Standard Deviation: 0.07831276745429881
1/1 [==============================] - 0s 38ms/step
Model with hidden_dim=32, num_layers=1, num_epochs=10, batch_size=64, lr=0.001
Average Accuracy: 0.5782051282051281
Standard Deviation: 0.10575174680667322
1/1 [==============================] - 0s 37ms/step
Model with hidden_dim=32, num_layers=1, num_epochs=10, batch_size=64, lr=0.01
Average Accuracy: 0.48461538461538467
Standard Deviation: 0.1290421395020184
1/1 [==============================] - 0s 36ms/step
Model with hidden_dim=32, num_layers=1, num_epochs=20, batch_size=32, lr=0.001
Average Accuracy: 0.5775641025641026
St